In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import json
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from pyecharts import options as opts 
from pyecharts.charts import *
from pyecharts.globals import ChartType, SymbolType,ThemeType
from pyecharts.components.table import Table


In [2]:
url="https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=nowConfirmStatis,provinceCompare"
r=requests.get(url)
dic=json.loads(r.text)
lst=[]
for i in dic['data']['provinceCompare'].keys():
    lst.append(i)
count=0
datadic={}
for i in lst:
    url=f"https://api.inews.qq.com/newsqa/v1/query/pubished/daily/list?province={i}"
    datadic[i]={}
    datadic[i]['日期']=[]
    datadic[i]['累计确诊']=[]
    r=requests.get(url)
    dic=json.loads(r.text)
    for j in dic['data']:
        datadic[i]['日期'].append(str(j['year'])+'.'+j['date'])
        datadic[i]['累计确诊'].append(j['confirm'])
lst1=pd.date_range('20200120',datetime.datetime.now().strftime("%Y%m%d")).strftime("%Y.%m.%d").tolist()
df=pd.DataFrame()
df['日期']=lst1
for i in datadic.keys():
    dfc=pd.DataFrame(datadic[i])
    df=pd.merge(df,dfc,on='日期',how='left')
    df.rename(columns={'累计确诊':f'{i}'},inplace=True)
df.fillna(method='bfill',axis=0,inplace=True)
df.fillna(method='ffill',axis=0,inplace=True)
df.drop_duplicates('日期','first',inplace=True)
df=df.set_index('日期')
for i in df.columns.to_list():
    df[i]=df[i].astype(float)

C:\Users\maple\AppData\Local\Temp\ipykernel_4756\1947756556.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop_duplicates except for the argument 'subset' will be keyword-only.
  df.drop_duplicates('日期','first',inplace=True)


In [3]:
df

,浙江,湖北,西藏,辽宁,云南,江西,重庆,青海,天津,福建,...,河北,黑龙江,内蒙古,四川,上海,陕西,河南,贵州,台湾,新疆
日期,,,,,,,,,,,,,,,,,,,,,
2020.01.20,62.0,270.0,0.0,64.0,70.0,3.0,6.0,0.0,14.0,82.0,...,65.0,80.0,11.0,90.0,203.0,56.0,1.0,3.0,0.0,0.0
2020.01.21,62.0,375.0,0.0,64.0,70.0,3.0,6.0,0.0,14.0,82.0,...,65.0,80.0,11.0,90.0,203.0,56.0,1.0,3.0,0.0,0.0
2020.01.22,62.0,444.0,0.0,64.0,70.0,3.0,9.0,0.0,14.0,82.0,...,65.0,80.0,11.0,90.0,203.0,56.0,5.0,3.0,1.0,0.0
2020.01.23,62.0,549.0,0.0,64.0,70.0,7.0,27.0,0.0,14.0,82.0,...,65.0,80.0,11.0,90.0,203.0,56.0,9.0,3.0,1.0,2.0
2020.01.24,62.0,729.0,0.0,64.0,70.0,18.0,58.0,1.0,14.0,82.0,...,65.0,80.0,11.0,90.0,203.0,56.0,32.0,4.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.06.14,3144.0,68399.0,1.0,1706.0,2157.0,1383.0,718.0,147.0,1985.0,3342.0,...,2005.0,2986.0,2054.0,2359.0,63132.0,3302.0,3183.0,185.0,3003501.0,1008.0
2022.06.15,3146.0,68399.0,1.0,1709.0,2157.0,1383.0,720.0,147.0,1985.0,3349.0,...,2007.0,2986.0,2067.0,2360.0,63145.0,3302.0,3183.0,185.0,3072432.0,1008.0
2022.06.16,3147.0,68340.0,1.0,1713.0,2157.0,1383.0,721.0,147.0,1985.0,3359.0,...,2007.0,2986.0,2073.0,2362.0,63150.0,3305.0,3183.0,185.0,3135565.0,1008.0


In [4]:
df.to_excel("china_province.xlsx",sheet_name="中国各省份累计疫情数据")

In [5]:
for i in lst:
    url=f"https://api.inews.qq.com/newsqa/v1/query/pubished/daily/list?province={i}"
    datadic[i]={}
    datadic[i]['日期']=[]
    datadic[i]['确诊新增']=[]
    r=requests.get(url)
    dic=json.loads(r.text)
    for j in dic['data']:
        datadic[i]['日期'].append(str(j['year'])+'.'+j['date'])
        datadic[i]['确诊新增'].append(j['newConfirm'])
lst1=pd.date_range('20200120',datetime.datetime.now().strftime("%Y%m%d")).strftime("%Y.%m.%d").tolist()
df2=pd.DataFrame()
df2['日期']=lst1
for i in datadic.keys():
    dfc=pd.DataFrame(datadic[i])
    df2=pd.merge(df2,dfc,on='日期',how='left')
    df2.rename(columns={'确诊新增':f'{i}'},inplace=True)
df2.fillna(0.0,axis=0,inplace=True)
df2.drop_duplicates('日期','first',inplace=True)
df2=df2.set_index('日期')
for i in df2.columns.to_list():
    df2[i]=df2[i].astype(float)

C:\Users\maple\AppData\Local\Temp\ipykernel_4756\3282530842.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop_duplicates except for the argument 'subset' will be keyword-only.
  df2.drop_duplicates('日期','first',inplace=True)


In [6]:
df2

,浙江,湖北,西藏,辽宁,云南,江西,重庆,青海,天津,福建,...,河北,黑龙江,内蒙古,四川,上海,陕西,河南,贵州,台湾,新疆
日期,,,,,,,,,,,,,,,,,,,,,
2020.01.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020.01.21,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020.01.22,0.0,69.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0
2020.01.23,0.0,105.0,0.0,0.0,0.0,4.0,18.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0
2020.01.24,0.0,180.0,0.0,0.0,0.0,11.0,30.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.06.14,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,14.0,2.0,18.0,1.0,0.0,0.0,66138.0,0.0
2022.06.15,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,7.0,...,2.0,0.0,13.0,1.0,13.0,0.0,0.0,0.0,68931.0,0.0
2022.06.16,1.0,1.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,10.0,...,0.0,0.0,6.0,2.0,5.0,3.0,0.0,0.0,63133.0,0.0


In [7]:
df2.to_excel("china_province_add.xlsx",sheet_name="中国各省份确诊新增数据")

In [8]:
url1="https://ncov.dxy.cn/ncovh5/view/pneumonia"
head={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"}
r=requests.get(url1,headers=head)
r.encoding=r.apparent_encoding
text=r.text
soup=BeautifulSoup(text)
datatotal=soup.select('body #getListByCountryTypeService2true')[0].text
datatotaljson=json.loads(re.findall(r'window.getListByCountryTypeService2.*?.=.(.*)\}catch',datatotal,re.S)[0])
for p in range(len(datatotaljson)):
    if datatotaljson[p]['countryFullName'] == 'China':
        location=p    
chinaurl=datatotaljson[location]['statisticsData']
r=requests.get(chinaurl)
datachinajson=json.loads(r.text)
with open("chinatotal.json","w",encoding="utf-8") as f:
    json.dump(datachinajson,f,indent=4)
chinadata=datachinajson['data']
columns=['累计确诊','累计确诊新增','累计治愈','累计治愈新增','现存确诊','现存确诊新增','日期','累计死亡','累计死亡新增','高风险地区数','中风险地区数','境外输入','境外输入新增']
df1=pd.DataFrame(chinadata)
df1.columns=columns
df1['日期']=df1['日期'].astype(str)
df1['日期']=df1['日期'].str[0:4]+"."+df1['日期'].str[4:6]+"."+df1['日期'].str[6:8]
df1=df1.set_index('日期')
df1.drop(columns=['高风险地区数','中风险地区数'],inplace=True)

In [9]:
df1

,累计确诊,累计确诊新增,累计治愈,累计治愈新增,现存确诊,现存确诊新增,累计死亡,累计死亡新增,境外输入,境外输入新增
日期,,,,,,,,,,
2020.01.19,217,217,25,25,188,188,4,4,0,0
2020.01.20,291,74,25,0,260,72,6,2,0,0
2020.01.21,440,149,25,0,406,146,9,3,0,0
2020.01.22,571,131,28,3,526,120,17,8,0,0
2020.01.23,835,264,34,6,776,250,25,8,0,0
...,...,...,...,...,...,...,...,...,...,...
2022.06.13,3496075,45319,295553,89,3181626,45121,18896,109,18930,35
2022.06.14,3562468,66393,295654,101,3247794,66168,19020,124,18953,23
2022.06.15,3631648,69180,295771,117,3316713,68919,19164,144,18975,22


In [10]:
df1.to_excel("chinatotal.xlsx",index=True)

In [11]:
line1 = Line(init_opts=opts.InitOpts()) 
line1.add_xaxis(df1.index.to_list())
line1.add_yaxis('新增确诊', df1['累计确诊新增'].tolist(), yaxis_index=1, color='#E3170D', is_smooth=True, symbol="none")
line1.add_yaxis('累计确诊', df1['累计确诊'].tolist(), yaxis_index=0, color='#FFFF00', is_smooth=True, symbol="none")
line1.add_yaxis('累计治愈', df1['累计治愈'].tolist(), yaxis_index=0, color='#00FF00', is_smooth=True, symbol="none")
line1.add_yaxis('累计死亡', df1['累计死亡'].tolist(), yaxis_index=1, color='#F0FFFF', is_smooth=True, symbol="none")
line1.add_yaxis('现存确诊', df1['现存确诊'].tolist(), yaxis_index=1, color='#8A2BE2', is_smooth=True, symbol="none")
line1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
line1.extend_axis(yaxis=opts.AxisOpts( axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))))
line1.set_global_opts(title_opts=opts.TitleOpts(title='中国疫情变化', pos_left = 'right', padding=[1,5], title_textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     xaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
                    ) 
line1.chart_id='5'
#line1.render_notebook()

In [12]:
bar1 = Bar(init_opts=opts.InitOpts()) 
bar1.add_xaxis(df1.index.to_list())

bar1.add_yaxis('新增确诊', df1['累计确诊新增'].tolist(), yaxis_index=1, color='#E3170D') 
bar1.add_yaxis('现存确诊', df1['现存确诊'].tolist(), yaxis_index=1, color='#F0FFFF') 
bar1.add_yaxis('累计确诊', df1['累计确诊'].tolist(), yaxis_index=0, color='#8A2BE2') 
bar1.add_yaxis('累计治愈', df1['累计治愈'].tolist(), yaxis_index=0, color='#FFFF00') 
bar1.add_yaxis('累计死亡', df1['累计死亡'].tolist(), yaxis_index=1, color='#00FF00') 

bar1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

bar1.extend_axis(yaxis=opts.AxisOpts( axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))))
bar1.set_global_opts(title_opts=opts.TitleOpts(title='中国疫情数据柱状图', pos_left = 'left', padding=[1,5], title_textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     xaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
                     legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     
                    ) 
bar1.chart_id='4'
#bar1.render_notebook()

In [13]:
table = (
         Table(opts.InitOpts())
         .add(headers=['日期','累计确诊','新增确诊','累计死亡','新增死亡'], 
              rows = [[df1.index.to_list()[-1],df1['累计确诊'].tolist()[-1],df1['累计确诊新增'].tolist()[-1],df1['累计死亡'].tolist()[-1],df1['累计死亡新增'].tolist()[-1]]
                     ],
              attributes = {"class": "fl-table","style":"background:;color:#FA5858"}
             )
)
table.chart_id='8'
#table.render_notebook()
# table.render('tables.html')

In [14]:
time1 = Timeline(init_opts = opts.InitOpts(
    chart_id = '2',
    #theme=ThemeType.CHALK,
    #theme=ThemeType.VINTAGE,
    #theme=ThemeType.PURPLE_PASSION,
    renderer = "canvas",
    #theme = "blue",
    #bg_color = "blue",
    ))
time1.add_schema(
    play_interval = 0.001,# 播放时间间隔；
)
for i in range(len(df1.index.to_list())):      
      outer_data_pair = [list(z) for z in zip(['累计确诊','累计治愈','累计死亡','境外输入'],[df1['累计确诊'].tolist()[i],df1['累计治愈'].tolist()[i],df1['累计死亡'].tolist()[i],df1['境外输入'].tolist()[i]])]
      inner_data_pair = [list(z) for z in zip(['确诊新增','治愈新增','死亡新增','境外输入新增'],[df1['累计确诊新增'].tolist()[i],df1['累计治愈新增'].tolist()[i],df1['累计死亡新增'].tolist()[i],df1['境外输入新增'].tolist()[i]])]

# 绘制饼图

      pie1 = Pie(init_opts=opts.InitOpts()) 
      pie1.add(' ', outer_data_pair, radius=['60%','80%'])
      pie1.add(' ', inner_data_pair, radius=[0,'40%'],rosetype='radius')
      pie1.set_global_opts(title_opts=opts.TitleOpts(title='全国数据玫瑰图', pos_right='1%', title_textstyle_opts=opts.TextStyleOpts(color='#FFFF99')), 
                        legend_opts=opts.LegendOpts(orient='vertical', pos_top='10%', pos_right='1%', textstyle_opts=opts.TextStyleOpts(color='#FFFF99')))
      pie1.set_series_opts(label_opts=opts.LabelOpts(position="outside", 
                                                formatter=" {per|{b}: {c}}",
                                                background_color="#eee",
                                                border_color="#aaa",
                                                border_width=1,
                                                border_radius=4,
                                                rich={
                                                      "per": {
                                                                  "color": "#eee",
                                                                  "backgroundColor": "#334455",
                                                                  "padding": [2, 4],
                                                                  "borderRadius": 2,
                                                                  },
                                                      },
                                                ))
      pie1.set_colors(['#EF9050', '#3B7BA9', '#6FB27C', '#FFAF34', '#00BFFF', '#FF2400','#40FF00'])
      #pie1.render_notebook()
      time1.add(pie1,df1.index.to_list()[i])
time1.chart_id='2'
#time1.render_notebook() 


In [15]:
pie2 = (
        Pie(init_opts=opts.InitOpts(chart_id = '9'))
        .add(
            series_name="各省确诊占比",
            data_pair=[list(z) for z in zip(df.columns.to_list(),[df.iloc[-1,k] for k in range(len(df.columns.tolist()))])],
            radius=["20%", "40%"],
            rosetype="area",
            label_opts=opts.LabelOpts(is_show=True),
        )
        .set_colors(['#EF9050', '#3B7BA9', '#6FB27C', '#FFAF34', '#00BFFF', '#FF2400','#40FF00'])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各省确诊占比"),
            legend_opts=opts.LegendOpts(orient='vertical', pos_top='10%', pos_right='1%', textstyle_opts=opts.TextStyleOpts(color='#FFFF99'),is_show=True)
            )       
        )
pie2.chart_id='9'
#pie2.render_notebook()

In [16]:
time = Timeline(init_opts = opts.InitOpts(
    chart_id = '1',
    #theme=ThemeType.CHALK,
    #theme=ThemeType.VINTAGE,
    #theme=ThemeType.PURPLE_PASSION,
    renderer = "canvas",
    #theme = "blue",
    #bg_color = "blue",
    ))
time.add_schema(
    play_interval = 0.001,# 播放时间间隔；
)
for i in df.index.to_list():
    map0 = (
        Map()
            .add("累计确诊人数",[list(z) for z  in zip(df.columns.to_list(),[df.loc[i,k] for k in df.columns.tolist()])], "china",
                 itemstyle_opts={
                "normal": {"areaColor": "#323c48", "borderColor": "#404a59"},
                "emphasis": {
                    "label": {"show": Timeline},
                    "areaColor": "rgba(255,255,255, 0.5)",
                },
            },
            is_map_symbol_show=False)
            .set_global_opts(
            title_opts=opts.TitleOpts(title="中国累计确诊人数变化"),
            visualmap_opts=opts.VisualMapOpts(max_=3000,min_=0, is_piecewise=False, range_color=["#CEECF5","#A9E2F3", "#81DAF5","#58D3F7","#2ECCFA","#FA5858"])) #'这里设置最大值,如果超过则不显示'
    )
    
    time.add(map0,i)
time.chart_id='1'
#time.render('chinaprovince.html')	# '在当前页面生成文件chinaprovince.html'
#time.render_notebook()

In [17]:
title = Pie().set_global_opts(title_opts=opts.TitleOpts(title="疫情大数据可视化展示", title_textstyle_opts=opts.TextStyleOpts(font_size=25, color='#00BFFF'), pos_top=0))
title.chart_id = '6'
#title.render_notebook()

In [18]:
subtitle = Pie().set_global_opts(title_opts=opts.TitleOpts(subtitle=("author:maple"),
                                                           subtitle_textstyle_opts=opts.TextStyleOpts(font_size=15, color='#00BFFF'),
                                                           pos_top=0,
                                                          )
                                )
subtitle.chart_id = '7'
#subtitle.render_notebook()

In [19]:
time2 = Timeline(init_opts = opts.InitOpts(
    chart_id = '3',
    #theme=ThemeType.CHALK,
    #theme=ThemeType.VINTAGE,
    #theme=ThemeType.PURPLE_PASSION,
    renderer = "canvas",
    #theme = "blue",
    #bg_color = "blue",
    ))
time2.add_schema(
    play_interval = 1000,# 播放时间间隔；
)
for i in df2.columns.tolist():
    #data=[list(z) for z  in zip(df2.index.to_list(),[day+1 for day in range(len(df2.index.to_list()))],[df2.loc[k,i] for k in df2.index.to_list()])]
    scatter = (
    Scatter()
    .add_xaxis(df2.index.to_list())
    .add_yaxis("确诊新增", [df2.loc[k,i] for k in df2.index.to_list()],symbol_size=10,color = '#2ECCFA')
    .set_global_opts(
        title_opts=opts.TitleOpts(title="疫情新增确诊散点图"),
        
        visualmap_opts=opts.VisualMapOpts(
            type_="size",  #映射大小
            max_=50,
            min_=10,
            pos_bottom = 50,pos_right = 0),
    )
)   
    time2.add(scatter,i)
time2.chart_id='3'
#time2.render_notebook()

In [20]:

page = Page(layout=Page.DraggablePageLayout, page_title='疫情数据大屏')
page.add(
         time,
         time1,
         time2,
         bar1,
         line1,
         title,
         subtitle,
         table,
         pie2
        )
page.render()

'd:\\python 大作业\\render.html'

In [21]:
Page.save_resize_html("render.html", cfg_file=r"shadow.json", dest="maple1.html");

In [ ]:
from bs4 import BeautifulSoup
with open("maple1.html", "r+", encoding='utf-8') as html:
    html_bf = BeautifulSoup(html, 'html.parser')
    #divs = html_bf.select('.chart-container')
    #divs[0]["style"] = "width:10%;height:10%;position:absolute;top:0;left:2%;"
    body = html_bf.find("body")
    body["style"] = "background-image: url(a10.png)"  # 背景颜色
    html_new = str(html_bf)
    html.seek(0, 0)
    html.truncate()
    html.write(html_new)